<a href="https://colab.research.google.com/github/mosdef-hub/CECAM-MoSDeF-Workshop/blob/main/solvated_surface_workflow/Solvated_Surface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MoSDeF-Signac Tutorial
## Solvated Surface Screening
---

## Tutorial summary
This tutorial aims to familiarize a molecular simulations researcher on the ways to
control a workflow managed by _[Signac.](https://docs.signac.io/en/latest/)_ This is
an important tool for accessing and storing workflow metadata such that the workflow
is standardized and [extensible](https://doi.org/10.1080/00268976.2020.1742938), as
part of the TRUE nature of computational research. The importance of managing a complex
workflow, such as one shown in the figure below, is becoming increasingly vital for
well organized research.

<img src="https://github.com/CalCraven/CECAM-MoSDeF-Workshop/blob/solvated-surface-wkflw/images/chemistry-workflow.png?raw=1" alt="Computational Chemistry Workflow from10.1080/00268976.2020.1742938" width="700"/>

The following workflow will show how to:
1. Calculate the interfacial structures of different three point water models. Water models include:
    - SPC/E
    - TIP3P-original
    - OPC
    - Note that 4-point water models are soon to be supported generally by virtual sites.
2. Build functional workflows that pull specifications from _Signac_ `job` criteria.
3. Run _HOOMD-blue_ simulations and process data in a standardized format for easy extension of the project aims.

## Learning Objectives
1. How to operate/set up a _[Signac](https://signac.io/)_ workflow.
2. Using _[mBuild](https://mbuild.mosdef.org/en/stable/)_ recipes and scripts for functional workflows.
3. How to pass [job metadata](https://docs.signac.io/projects/core/en/latest/api.html#the-job-class) to a simulation process.
4. _[HOOMD-blue](https://hoomd-blue.readthedocs.io/en/v4.1.0/)_ simulations.
5. Data processing with _[Freud](https://freud.readthedocs.io/en/latest/)_.

## Tutorial Contents
0. Google Colab Setup and import packages
1. Initialize _Signac_ `Project`
    1. Exercise 1. Adding project jobs by statepoints
    
2. Generate _mBuild_ Structures
    1. Exercise 2. Pack water at higher initial density
    
3. Apply Force Fields
    1. Exercise 3a. Identify bondtype and angletype expressions used.
    1. Exercise 3b. Determine the number of unique bond types used.
    
4. Run _HOOMD-blue_ Simulations

5. Analyze/Record Results
    1. Exercise 5. Load data into [_MDAnalysis_](https://www.mdanalysis.org/)

### Software Stack Setup
After running the cell below the kernel will restart -- This is necessary for conda dependencies, but you'll need to wait for that kernel restart before running the second cell. Expect this to take about 5 minutes.

### Working with Google Colab
There are two types of output in these Colab notebooks that can be a little tricky:

1. If the output is very long, for example from the mamba command in the second cell, scrolling past the output can feel onerous. In this case, scrolling up and down in the narrow grey area between the sidebar menu and the cells can help you navigate.
2. If the output is a visualization of a molecule or simulation configuration, scrolling up or down will zoom in or out if the cursor is over the visualization. In these cases, take some care to scroll outside of the visualization.
3. To run a cell, either click the run button (right facing triangle) or hit shift + enter


## 0. Google Colab Setup
---
Run the two next panes only if you are using Google Collab for this notebook.

In [ ]:
# Note: Run this cell first and by itself.
# The kernel will be restarted after this step. Rerun this cell if that happens.
# There might be an error pops up stating the session crashed
# for an unknown reason, but that is expected. You do not have to rerun in that case.

!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

In [ ]:
# import condacolab
# condacolab.check()

!conda install mamba
!mamba install mbuild foyer hoomd gsd signac mdanalysis freud py3dmol fresnel openbabel gmso freud

!git clone https://github.com/kierannp/surface_coatings.git
%cd surface_coatings
!git fetch origin colab-installation
!git checkout colab-installation
!pip install .
%cd ..
!git clone https://github.com/mosdef-hub/FOMMS-MoSDeF-Workshop.git
%cd FOMMS-MoSDeF-Workshop/workflows/solvated_surface_workflow/
!pwd

## 0. Import packages
---

In [ ]:
# Import Libraries
import itertools
import os

import signac
import mbuild as mb
import gmso
from gmso.parameterization import apply
import gsd
import unyt as u
import numpy as np

from surface_coatings.monolayer import Monolayer
from surface_coatings.solvated_monolayer import SolvatedMonolayer
from surface_coatings.surfaces.silica_interface import SilicaInterface
from surface_coatings.surfaces.silica_interface_carve import SilicaInterfaceCarve
from surface_coatings.chains.alkylsilane import Alkylsilane

import warnings
warnings.filterwarnings("ignore")

## 1. Initialize Signac Project
---
From the _Signac_ Documentation:
"The signac framework supports researchers in managing project-related data with a well-defined indexable storage layout for data and metadata. This streamlines post-processing and analysis and makes data collectively accessible. The signac framework aims to help make computational research projects Transparent, Reproducible, Usable by others, and Extensible (TRUE) [TGM+20], a set of principles put forth by the MoSDeF Collaboration [CMI+21]."</br></br>

<figure>
    <img src="https://github.com/CalCraven/CECAM-MoSDeF-Workshop/blob/solvated-surface-wkflw/images/signac.png?raw=1" alt="Signac Framework replicated from https://doi.org/10.1016/j.commatsci.2018.01.035" width="700"/>
    <figcaption><center>"Signac Graphical Abstract (Adorf et al. 2017 10.1016/j.commatsci.2018.01.035)"</center>
    </figcaption>
</figure>


We will use _Signac_ to define some project parameters. The parameters are stored in the _Signac_ `job` class that are operated on in the study. The following function, `grid`, is a standard way to generate all combinations of each statepoint key-value pairs, defined in the dictionary variable `sp_gridDict`.

- Run the following cell and evaluate the `project` that is generated.

In [ ]:
def grid(gridspec):
    """Yields the Cartesian product of a `dict` of iterables.

    The input ``gridspec`` is a dictionary whose keys correspond to
    parameter names. Each key is associated with an iterable of the
    values that parameter could take on. The result is a sequence of
    dictionaries where each dictionary has one of the unique combinations
    of the parameter values.
    """
    for values in itertools.product(*gridspec.values()):
        yield dict(zip(gridspec.keys(), values))

project = signac.init_project("./solvated_surface_project") # name and path of project
# generate statepoints:
sp_gridDict = {
    "water_model":["spce", "tip3p"], # forcefield to use
    "temperature":[298.15],  # K
    "chain_density":[3], # chains/nm
    "chain_length":[4], # n_carbons
    "solvent_box_height": [2], #nm
    "seed":[314159], # for random configurations
}

for sp in grid(sp_gridDict):
    print("Initializing job", sp)
    project.open_job(sp).init()
project

Initializing the `project` generates a local directory to store information generated by the workflow. Here are a list of some of the files.

In [ ]:
!ls -a solvated_surface_project/

The workspace is a directory where the individual jobs that were initialized above are stored. These are differentiated by unique hashes generated from each statepoint so there's no risk of accidentally overwriting data from alternative jobs.

Let's look at the `signac_statepoint.json` file in one of our jobs.

In [ ]:
!cat solvated_surface_project/workspace/abc17508672efbf5f8b8afd4ad096ab8/signac_statepoint.json

Nice work! You should see something that looks like a python dictionary with all of the information that was passed in the above cells. For more information on how to access this job metadata, see the following [_Signac_ tutorials.](https://docs.signac.io/en/latest/tutorial.html#interacting-with-the-signac-project)

## Exercise 1: Try to add a new job to the workspace.
Choose:
- Temperature of 310 K
- 3 chains/nm\*\*2
- 15 carbon chain length.

Validate that the `job` directory is added properly.


In [ ]:
project = signac.get_project("./solvated_surface_project/")
jobSP = { #initialize a dictionary statepoint
    "water_model":"spce", # forcefield to use
    "temperature":???,  # K
    "chain_density":???, # chains/nm**2
    "chain_length":???, # n_carbons
    "solvent_box_height": 2, #nm
    "seed":42901423,
}
project.???.????() # open and initialize the jobSP dictionary you made
project # how many jobs are there now?

### <font color="red"><b>Exercise 1 Answer</b></font>

<details>
  <summary>Click once to hide/unhide the answer!</summary>

    project = signac.get_project("./solvated_surface_project/")
    jobSP = { #initialize a dictionary statepoint
        "water_model":"spce", # forcefield to use
        "temperature":310,  # K
        "chain_density":3, # chains/nm**2
        "chain_length":15, # n_carbons
        "solvent_box_height": 2, #nm
        "seed":42901423,
    }
    project.open_job(jobSP).init()
    project

</details>

## 2. Generate mBuild Structures
---

Next we will use _mBuild_ (which you're already an expert in) to build up the individual components of our surface. The key components in this hierarchical structure are:
- The surface, made from silica. This will be loaded via premade recipe in the [coated_surfaces](https://github.com/daico007/surface_coatings/tree/main) repository.
- The chains to attach to the surface. These are alkylsilane chains, built using the [Polymer Builder](https://github.com/mosdef-hub/mbuild/blob/main/mbuild/lib/recipes/polymer.py) recipe in `mBuild`.
- The water atoms, which are simple three site HOH molecules, made from the [SMILES string](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html).

In [ ]:
job_compoundDict = {} # dictionary to store job compounds

# load the water
water = mb.load("O", smiles=True)
water.name = "water"
solvent_density = 0.35 * u.g/u.cm**3 # calculate density in g/cm^3
atomic_density = solvent_density / (u.amu.to("g")) / water.mass # convert density to atoms/nm^3
print(f"Water density is: {atomic_density.to('nm**-3'):.0f}")

for job in project: #iterate over all jobs using a simple for loop on the `project`
    print(f"Building up surfaces for job:{job.id}. Run time is about 3 minutes for each job with energy minimization (see commented out code below).")
    chain = Alkylsilane(chain_length=job.sp.chain_length) #create a chain
    surface = SilicaInterfaceCarve(thickness=1.2)
    surface_area = surface.box.Lx * surface.box.Ly
    n_chains=int(job.sp.chain_density*surface_area)
    graft_pattern = mb.Random2DPattern(n_chains, seed=job.sp.seed)
    monolayer = Monolayer(surface=surface, pattern=graft_pattern, chains=chain, n_chains=n_chains)
    monolayer.name = "surface" # set the name for later atomtyping
    n_waters = int(surface_area*job.sp.solvent_box_height*atomic_density.to("nm**-3").value)
    print(f"Adding {n_waters:d} waters to the system.")
    solvated_monolayer = SolvatedMonolayer(
        monolayer=monolayer, solvent=water,
        n_solvents=n_waters, solvent_box_height=job.sp.solvent_box_height)
    print(f"Currently Energy minimizing, \n  - expect to take ~5 min  or comment out next 5 lines of code.")
    children = [surface] # fix surface
    import time
    start = time.perf_counter()
    solvated_monolayer.energy_minimize(fixed_compounds=children, steps=100) # relax chains and water
    print(f"Energy minimized structure using UFF in {int(time.perf_counter()-start):d}s\n\n")
    job_compoundDict[job] = { # store the saved components for visualization
        "water":water,
        "chain": chain,
        "surface":surface,
        "solvated_monolayer":solvated_monolayer
    }


Let's visualize the monolayer that was built.

In [ ]:
solvated_monolayer.visualize()

## Exercise 2: Increase the water packing density.
Try to pack the water at a higher density. Find the line
`solvent_density = 0.35 * u.g/u.cm**3 # g/cm^3`
and replace `0.35` with `0.99` for a better packed system. **Note that this will take longer to pack at higher densities.**

Also, there looks like there's a gap in between the water box and the top of the surface. Fix that by adding


`solvated_monolayer.children[1].translate([0,0,-0.3])`


To move just the water molecules down by 0.3 nm. This should be added after the line


`solvated_monolayer = SolvatedMonolayer(monolayer=monolayer, solvents=water, n_solvents=n_waters, solvet_box_height=job.sp.solvent_box_height)`

### <font color="red"><b>Exercise 2 Answer</b></font>
<details>
  <summary>Click once to hide/unhide the answer!</summary>

    job_compoundDict = {} # dictionary to store job compounds

    # load the water
    water = mb.load("O", smiles=True)
    water.name = "water"
    solvent_density = 0.99 * u.g/u.cm**3 # calculate density in g/cm^3 <------- ADDED CODE HERE
    atomic_density = solvent_density / (u.amu.to("g")) / water.mass # convert density to atoms/nm^3
    print(f"Water density is: {atomic_density.to('nm**-3'):.0f}")

    for job in project: #iterate over all jobs using a simple for loop on the `project`
        print(f"Building up surfaces for job:{job.id}. Run time is about 3 minutes for each job with energy minimization (see commented out code below).")
        chain = Alkylsilane(chain_length=job.sp.chain_length) #create a chain
        surface = SilicaInterfaceCarve(thickness=1.2)
        surface_area = surface.box.Lx * surface.box.Ly
        n_chains=int(job.sp.chain_density*surface_area)
        graft_pattern = mb.Random2DPattern(n_chains, seed=job.sp.seed)
        monolayer = Monolayer(surface=surface, pattern=graft_pattern, chains=chain, n_chains=n_chains)
        monolayer.name = "surface" # set the name for later atomtyping
        n_waters = int(surface_area*job.sp.solvent_box_height*atomic_density.to("nm**-3").value)
        print(f"Adding {n_waters:d} waters to the system.")
        solvated_monolayer = SolvatedMonolayer(
            monolayer=monolayer, solvent=water,
            n_solvents=n_waters, solvent_box_height=job.sp.solvent_box_height)
        solvated_monolayer.children[1].translate([0,0,-0.3]) # <----------- ADDED CODE HERE
        #print(f"Currently Energy minimizing, \n  - expect to take ~12 min  or comment out next 5 lines of code.")
        #children = [surface] # fix surface
        #import time
        #start = time.perf_counter()
        #solvated_monolayer.energy_minimize(fixed_compounds=children, steps=500) # relax chains and water
        #print(f"Energy minimized structure using UFF in {int(time.perf_counter()-start):d}s\n\n")
        job_compoundDict[job] = { # store the saved components for visualization
            "water":water,
            "chain": chain,
            "surface":surface,
            "solvated_monolayer":solvated_monolayer
        }
     
</details>

## 3. Apply Force Fields
---

We must now use force fields that define the potential equations of this system. The force fields are stored on disk as XML files. These define all of the forces needed for the simulation. Multiple force fields can be applied to different molecules in the _mBuild_ `compound` but be certain that these force fields are compatible (i.e. same mixing rule, 1-4 scaling, cutoffs). Also, while these force fields are stored in a generalized format, not all simulation engines can support all force field types, so keep that in mind when selecting a force field to use.

- We can iterate through the `jobs`, grab the `solvated_surface`, load the water force field specified in `job.sp.water_model` from the `xmls/` directory in this project, and create our parameterized _GMSO_ `Topology`.

- First though, let's glance at the syntax for the `apply` step.

In [ ]:
from gmso.parameterization import apply
help(apply)

- You should see that the `forcefields` argument can be passed as a dictionary. Since we added the names `water` and `surface` to the _mBuild_ `comopounds` we generated above, these labels are present and allow us to specify which component of the `topology` will be matched to each forcefield.

- Now let's apply the [OPLS-AA force field](https://pubs.acs.org/doi/10.1021/ja9621760) to the surface and the specified water force field from the `job.sp.water_model`.

In [ ]:
surfaceFF = gmso.ForceField("xmls/oplsaa.xml") # load a general opls all atom forcefield from the directory

for job in project:
    ff_path = os.path.join("xmls", job.sp.water_model+".xml")
    waterFF = gmso.ForceField(ff_path)
    top = job_compoundDict[job]["solvated_monolayer"].to_gmso()
    top.identify_connections() # generate bonds, angles, and dihedrals
    import time
    start = time.perf_counter()
    parameterized_top = apply(
        top, forcefields={"surface": surfaceFF, "water":waterFF}, # matching dictionary to compound labels
        remove_untyped=True
    )
    print(
        f"It took {time.perf_counter()-start:.2f}s to parameterize {top.n_sites} sites "
        + f"and {top.n_connections} connections."
    )
    job_compoundDict[job]["parameterized_top"] = parameterized_top

parameterized_top

## Exercise 3a: Identify the aspectes of the parameterized topology
What is the bonded equation that is being used from both of our forcefields?

To identify this, we need to interrogate the `parameterized_topology` object. Components are stored in IndexedSets. These can be accessed via list indexing.

In [ ]:
parameterized_top.bonds[?].???_type.expression # access index 0 and print then access the bond_type.expression
# this can also be done with angles, dihedrals, and impropers

### <font color="red"><b>Exercise 3a Answer</b></font>

<details>
  <summary>Click once to hide/unhide the answer!</summary>

    parameterized_top.bonds[0].bond_type.expression # access index 0 and print then access the bond_type.expression
    # this can also be done with angles, dihedrals, and impropers
    parameterized_top.angles[0].angle_type.expression
    # Answer should look something like this:
    # 0.5𝑘(𝑟−𝑟𝑒𝑞)2
</details>

## Exercise 3b: What is the number of unique bond types in the topology?

To identify that, use a filter to look for bond_types with a unique pair of atom classes

- Potential filters allow the user to specify what is `unique` about the type. These give a flexible set of methods to get the unique types in the system. For instance, there may be a set with unique parameters. Maybe you want to look at the set with unique full potential expressions. In this case, we will sort by the `unique names of the atom classes` that make up the bonded atoms.

In [ ]:
from gmso.core.views import PotentialFilters

pfilter = PotentialFilters.UNIQUE_NAME_CLASS
print(f"There are {len(parameterized_top.bond_types(filter_by=???????))} uniquely named bond types")
display(PotentialFilters.__dict__) # try a different filter from the PotentialFilters class

### <font color="red"><b>Exercise 3b Answer</b></font>
<details>
  <summary>Click once to hide/unhide the answer!</summary>
  
    from gmso.core.views import PotentialFilters

    pfilter = PotentialFilters.UNIQUE_NAME_CLASS
    print(f"There are {len(parameterized_top.bond_types(filter_by=???????))} uniquely named bond types")
    display(PotentialFilters.__dict__) # try a different filter from the PotentialFilters class
</details>

## 4. Run _HOOMD-blue_ Simulations
---

Now let's run our [_HOOMD-blue_](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulations for each job. We create a generalized `run_hoomd` function here which accepts whatever we write out from the _GMSO_ `Topology` `parameterized_top`.

- Generate the input structures to _hoomd_, the snapshot and the forcefield.
- Run a short example simulation
- \*\*It's noted that a longer simulation procedure is also provided, but will not be run as part of this tutorial as the simulation needs to be run on HPC. However, an output trajectory from such a simulation is provided for data analysis.

In [ ]:
from gmso.external import to_hoomd_snapshot, to_hoomd_forcefield
import hoomd
import numpy as np

"""Simulation configuration and runtime parameters."""
job = project.open_job(id="abc17508672efbf5f8b8afd4ad096ab8")

base_units = { # this unit sytems lets hoomd non-dimensionalize all parameters in the forcefield
    "mass": u.g / u.mol,
    "length": u.nm,
    "energy": u.kJ / u.mol,
}

top = job_compoundDict[job]["parameterized_top"]

gmso_snapshot, snapshot_base_units = to_hoomd_snapshot(
    top, base_units=base_units
)
gmso_forces, forces_base_units = to_hoomd_forcefield(
    top,
    r_cut=1.2, #u.nm
    base_units=base_units,
    pppm_kwargs={"resolution": (64,64,64), "order": 7},
)

In [ ]:
from hoomd_runners import example_run
job = project.open_job(id="abc17508672efbf5f8b8afd4ad096ab8")
top = job_compoundDict[job]["parameterized_top"]
example_run(job, top, gmso_snapshot, gmso_forces, dt=0.0001)

To run the full simulation, change the cell below to code and execute it. This will not execute without the surface minimization step commented out when building the monolayers in _mBuild_. </br>
**\*\*Note that this will be a long simulation (~15 hours) if run on Google Colab or locally.**

```python
from hoomd_runners import nvt_run
"""
## Please uncomment out the energy minimization lines in the above cells
## when building the jobs to ensure the initial state is low enough energy
# NOT HERE, RUN THESE NEXT 3 LINES ABOVE
# children = [surface] # fix surface
# solvated_monolayer.energy_minimize(fixed_compounds=children) # relax chains and water
# print(f"Energy minimized structure using UFF")
"""
job = project.open_job(id="64449306e141308096f2860598c7990a") # pick a job to run, or loop over jobs in project
top = job_compoundDict[job]["parameterized_top"]
base_units = { # this unit sytems lets hoomd non-dimensionalize all parameters in the forcefield
    "mass": u.g / u.mol,
    "length": u.nm,
    "energy": u.kJ / u.mol,
}

gmso_snapshot, snapshot_base_units = to_hoomd_snapshot(
    top, base_units=base_units
)
gmso_forces, forces_base_units = to_hoomd_forcefield(
    top,
    r_cut=0.6,
    base_units=base_units,
    pppm_kwargs={"resolution": (64,64,64), "order": 7},
)
nvt_run(job, top, gmso_snapshot, gmso_forces)
```

## 5. Analyze/Record Results
---

- Plot the simulation energies.
- View the trajectory.

In [ ]:
import gsd.hoomd
sample_project = signac.get_project("./sample-project")
job = sample_project.open_job(id="abc17508672efbf5f8b8afd4ad096ab8") # pick the id of the job you would want to test

data = gsd.hoomd.read_log(job.fn("trajectory-nvt.gsd"))
timestep = data['configuration/step']
potential_energy = data[
    'log/md/compute/ThermodynamicQuantities/potential_energy' # TODO: data won't load, error in gsd writing
]

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(5, 3))
ax.plot(timestep, potential_energy)
ax.set_xlabel(r'Timestep ($fs$)')
ax.set_ylabel(r'Potential energy ($kj/mol$)')
fig.show()

View the trajectory using [_Fresnel_](https://fresnel.readthedocs.io/en/latest/). Alternative methods to open a `HOOMD-blue` trajectory are [_VMD_](https://github.com/mphowardlab/gsd-vmd) (via plugin) or [_Ovito_](https://www.ovito.org/about/) (native support), or write out .DCD files during the simulation.

In [ ]:
# view the trajectory
sample_project = signac.get_project("./sample-project")
job = sample_project.open_job(id="abc17508672efbf5f8b8afd4ad096ab8")
traj = gsd.hoomd.open(job.fn("trajectory-nvt.gsd"))

from hoomd_runners import render_movie
render_movie(traj, job) # pass in indices=np.arange(0, 3600) to see surface only

## Exercise 5.
Load the trajectories into _MDAnalysis_. Simulation analysis is then accessible for the trajectory.
- MDAnalysis uses a .gsd file for the trajectory.
- It can take a .gro file for the topology.
- Other details on loading an [_MDAnalysis_](https://userguide.mdanalysis.org/stable/reading_and_writing.html) `Universe` can be found here.

In [ ]:
from MDAnalysis import Universe

sample_project.????_???(id="9a2a12d8920bfc96dbb933bea2ee9b81") #load the job
# in this case, a gro file is nice to have for the analysis.
# This can always be written by using parameterized_top.save(job.fn("init.gro"))
universe = Universe(job.??("trajectory-nvt.gsd"), top=job.??("init.gro")) # use the fn to grab files in the job
universe.trajectory # view the universe trajectory

### <font color="red"><b>Exercise 5 Answer</b></font>
<details>
  <summary>Click once to hide/unhide the answer!</summary>
  
    from MDAnalysis import Universe
    sample_project.open_job(id="9a2a12d8920bfc96dbb933bea2ee9b81") #load the job
    # in this case, a gro file is nice to have for the analysis.
    # This can always be written by using parameterized_top.save(job.fn("init.gro"))
    universe = Universe(job.fn("trajectory-nvt.gsd"), top=job.fn("init.gro")) # use the fn to grab files in the job
    universe.trajectory # view the universe trajectory
</details>